# 1 数据探索

# 2 特征工程

# 3 模型融合
模型融合也即是集成学习。


## voting
Voting（投票机制）分为硬投票和软投票两种，其原理采用少数服从多数的思想，此方法可用于分类问题。

### 软投票示例

**导入相关包**

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from mlxtend.data import iris_data
from mlxtend.plotting import plot_decision_regions


**EnsembleVoteClassifier实现**

In [ ]:
# Initializing Classifiers
clf1 = LogisticRegression(random_state=0, solver='lbfgs', multi_class='auto')
clf2 = RandomForestClassifier(random_state=0, n_estimators=100)
clf3 = SVC(random_state=0, probability=True, gamma='auto')
eclf = EnsembleVoteClassifier(clfs=[clf1, clf2, clf3], weights=[2, 1, 1], voting='soft')

# Loading some example data
X, y = iris_data()
X = X[:, [0, 2]]

# plotting decision regions
gs = gridspec.GridSpec(1, 4)
fig = plt.figure(figsize=(16, 4))

for clf, lab, grd, in zip([clf1, clf2, clf3, eclf],
                          ['Logistic Regression', 'Random Forest', 'RBF kernel SVM', 'Ensemble'],
                          itertools.product([0, 1], repeat=2)):
    clf.fit(X, y)
    ax = plt.subplot(gs[0, grd[0] * 2 + grd[1]])
    fig = plot_decision_regions(X=X, y=y, clf=clf, legend=2)
    plt.title(lab)
plt.show()

-------
**代码说明**

1. itertools.product([0, 1], repeat=2)：产生[0,0][0,1][1,0][1,1];
-------

### 参考资料
1. 《天池赛题解析》P103页；

## averaging
averaging用于回归问题，原理是将模型结果的平均值作为最终预测值。

## ranking

## 3.1 blending

### Blending 流程
1. 将数据划分为训练集和测试集(test_set)，其中训练集需要再次划分为训练集(train_set)和验证集(val_set)；
2. 创建第一层的多个模型，这些模型可以使同质的也可以是异质的（这里的创建是初始化几个模型实例的意思，等待下一步训练）；
3. 使用train_set训练步骤2中的多个模型，然后用训练好的模型预测val_set和test_set得到val_predict, test_predict1；这一步的做法其实是把第一层训练得到的模型当成特征转换工具：$\Phi(x)=(g_1(x),...,g_k(x))$.
4. 创建第二层的模型,使用val_predict作为训练集训练第二层的模型的，即找到最好的组合系数$\alpha_{i}$；
5. 使用第二层训练好的模型对第二层测试集test_predict1进行预测，该结果为整个测试集的结果



### Blending 图解

<br></br>
<center><img src="./img/2019052109512454.png" width="600" hegiht="" ></center>
<center><br>bagging流程图</br></center>
<br></br>

## 3.4 stacking

### Stacking 流程
以下图为例：
1. 有训练集和测试集两组数据，将训练集分成5份：train1,train2,train3,train4,train5;
2. 选定基模型。这里假定选择了xgboost、lightgbm、randomforest作为基模型。比如xgboost部分，依次用train1,train2,train3,train4,train5做为验证集，其余4份作为训练集，然后5折交叉进行模型训练，再在测试集上进行预测（5个模型的预测结果平均）。这样会在训练集上由xgboost模型训练出来的5份prediction和在测试集上得到的B1，然后将5份prediction纵向重叠合并起来得到A1.lightgbm模型和randforest模型部分相同；
3. 在三个基模型训练完毕后，将三个模型在训练集上的预测值分别作为3个“特征”A1,A2,A3，然后使用LR模型进行训练并建立LR模型；
4. 使用训练好的LR模型，在色个基模型的测试集上预测得到的“特征”（B1,B2,B3）的基础上进行预测，得出最终的预测类别或概率。  

**说明：**  
在做stacking的过程中，如果将第一层模型的预测值和原始特征合并加入第二层模型训练中，则可以使模型的效果更好，还可以防止模型过拟合。

<br></br>
<center><img src="./img/截屏2021-04-04 下午1.01.25.png" width="600" hegiht="" ></center>
<center><br>stacking流程图</br></center>
<br></br>

## 3.2 bagging

### 3.2.1 Bagging算法  
Bagging算法，又称装袋算法，是机器学习领域的一种集成学习算法。最初由Leo Breiman于1994年提出。之所以被称为装袋法，是因为它采用了一种有放回的抽样方法来生成训练数据。通过多轮有放回的对初始训练集进行随机采样，多个训练集被并行化生成，对应可训练出多个基学习器（基学习器间不存在强依赖关系），再将这些基学习器结合，构建出强学习器。其本质是引入了样本扰动，通过增加样本随机性，达到降低方差的效果。

<br></br>
<center><img src="./img/9210113-8ae1f9ba634caf4f.png" width="600" hegiht="" ></center>
<center><br>bagging流程图</br></center>
<br></br>

因为 bagging 方法可以减小过拟合，所以通常在强分类器和复杂模型上使用时表现的很好（例如，完全生长的决策树，fully developed decision trees），相比之下 boosting 方法则在弱模型上表现更好（例如，浅层决策树，shallow decision trees）。


### 3.2.2 执行步骤
假设有1000个样本，70%的样本作为训练集，30%的样本作为测试集。
- STEP1：从训练集700条里样本随机抽出500条样本，用来训练，于是生成了一个基学习器。
- STEP2：然后有放回的再从700条样本在随机抽出500条样本，用来训练，于是又生成了一个基学习器
- STEP3：假设以步骤2的方式随机抽取100次，这样就生成了100个基学习器了
- STEP4：用这100个基学习器对测试集的每一个样本都进行测试，然后通过投票的方式决定测集样本的预测结果。如果一个样本被这100个基学习器投票，即预测分类，被分为1的票数有80票，被分为0的有20票，显然，这个样本的预测结果为1。


### 3.2.3 使用sklearn库实现Bagging方法
**BaggingClassifier**  
决策树是一个可读性很强、分类快，同时不需要对数据进行归一化还有缩放的处理。但是决策树有个不足的地方就是非常容易过拟合，所以必须要采取一些方法进行剪枝。而bagging方法的出现，可以完美地解决了决策树过拟合的问题，同时bagging的使用也会使分类器分类效果得到了显著的提高。  
在 scikit-learn 中，bagging 方法使用统一的 BaggingClassifier 元估计器.

**导入相关包**

In [ ]:
#-*- coding:utf-8 -*-

from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

**载入iris数据集**

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target

# print('feature=',X)
# print('target=',y)

**bagging融合**

In [ ]:
def BaggingMethod(X, y):
    '''
    Bagging方法实现分类
    INPUT -> 特征, 分类标签
    '''
    scaler = StandardScaler() # 标准化转换
    scaler.fit(X)  # 训练标准化对象
    traffic_feature= scaler.transform(X)   # 转换数据集
    feature_train, feature_test, target_train, target_test = train_test_split(X, y, test_size=0.3, random_state=0)
    tree = DecisionTreeClassifier(criterion='entropy', max_depth=None)

    clf = BaggingClassifier(base_estimator=tree, 
                            n_estimators=500,   # 生成500个决策树
                            max_samples=1.0,
                            max_features=1.0,
                            bootstrap=True,
                            bootstrap_features=False,
                            n_jobs=1,
                            random_state=1)
    clf.fit(feature_train, target_train)

    # 模型测试
    predict_results = clf.predict(feature_test)
    print(accuracy_score(predict_results, target_test))
    conf_mat = confusion_matrix(target_test, predict_results)
    print(conf_mat)
    print(classification_report(target_test, predict_results))

    return clf

**主程序**

------

**参数说明：**

1. base_estimator：基本模型，None默认为决策树；
2. n_estimators=500：生成500个决策；
3. max_samples : int或float，可选（默认值= 1.0），从X抽取以训练每个基本模型（estimator）的样本数，如果为int，则抽取样本 max_samples，如果float，则抽取本 max_samples * X.shape[0]。
4. max_features：int或float，可选（默认值= 1.0）从X绘制以训练每个基本估计量的要素数量。如果为int，则绘制特征 max_features。如果是浮动的，则绘制特征 max_features * X.shape[1]。
5. bootstrap : 布尔值，可选（默认= True）是否抽取样本进行替换。如果为False，则执行不替换的采样。
6. n_jobs : int或None（可选）（默认为None）fit和 并行运行的作业数predict。None除非joblib.parallel_backend上下文中，否则表示1 。-1表示使用所有处理器。
7. random_state : 整型，RandomState实例或无，可选（默认值：无）如果为int，则random_state是随机数生成器使用的种子；否则为false。

------

In [ ]:
if __name__ == '__main__':
    model = BaggingMethod(X, y)

### 3.2.4 参考资料
1. 机器学习中的集成方法（2）--Bagging（装袋法）（https://www.jianshu.com/p/fa212d4391f7）;
2. scikit-learn中BaggingClassifier参数（https://blog.csdn.net/Asdas_/article/details/104055529）；

## 3.3 boosting

# 4 模型验证

KFold方法  k折交叉验证  
交叉验证的基本思想是这样的：将数据集分为k等份，对于每一份数据集，其中k-1份用作训练集，单独的那一份用作测试集。

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [111, 222], [333, 444]])
y = np.array([1, 2, 3, 4, 5, 6])

kf = KFold(n_splits=3)
for train_index, test_index in kf.split(X):
    print('train_index', train_index, 'test_index', test_index)
    train_X, train_y = X[train_index], y[train_index]
    test_X, test_y = X[test_index], y[test_index]

RepeatedKFold  p次k折交叉验证  
一份数据分成k份也是随机分的，KFold直进行一次随机分割，而RepeatedKFold进行n_repeats次随机分割，相当于进行n_repeats次KFold。

In [ ]:
from sklearn.model_selection import RepeatedKFold
import numpy as np

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [111, 222], [333, 444]])
y = np.array([1, 2, 3, 4, 5, 6])

kf = RepeatedKFold(n_splits=3, n_repeats=2, random_state=0)
for train_index, test_index in kf.split(X):
    print('train_index', train_index, 'test_index', test_index)

**GridSearchCV概念呢**  
GridSearchCV的名字其实可以拆分为两部分，GridSearch和CV，即网格搜索和交叉验证。这两个名字都非常好理解。网格搜索，搜索的是参数，即在指定的参数范围内，按步长依次调整参数，利用调整的参数训练学习器，从所有的参数中找到在验证集上精度最高的参数，这其实是一个训练和比较的过程。

　　GridSearchCV可以保证在指定的参数范围内找到精度最高的参数，但是这也是网格搜索的缺陷所在，他要求遍历所有可能参数的组合，在面对大数据集和多参数的情况下，非常耗时。

**网络搜索原理**  
下面用代码解释一下原理。

In [ ]:
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

iris = load_iris()
X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target,random_state=0)
print("Size of training set:{} size of testing set:{}".format(X_train.shape[0],X_test.shape[0]))

####   grid search start
best_score = 0
for gamma in [0.001,0.01,0.1,1,10,100]:
    for C in [0.001,0.01,0.1,1,10,100]:
        svm = SVC(gamma=gamma,C=C)#对于每种参数可能的组合，进行一次训练；
        svm.fit(X_train,y_train)
        score = svm.score(X_test,y_test)
        if score > best_score:#找到表现最好的参数
            best_score = score
            best_parameters = {'gamma':gamma,'C':C}
####   grid search end

print("Best score:{:.2f}".format(best_score))
print("Best parameters:{}".format(best_parameters))

网格搜索算法使用每组超参数训练模型并挑选验证集误差最小的超参数组合，也就是每次repeat，只产生一个验证误差。

In [ ]:
from sklearn.model_selection import GridSearchCV

#把要调整的参数以及其候选值 列出来；
param_grid = {"gamma":[0.001,0.01,0.1,1,10,100],
             "C":[0.001,0.01,0.1,1,10,100]}
print("Parameters:{}".format(param_grid))

grid_search = GridSearchCV(SVC(),param_grid,cv=5, verbose=2) #实例化一个GridSearchCV类
X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target,random_state=10)
grid_search.fit(X_train,y_train) #训练，找到最优的参数，同时使用最优的参数实例化一个新的SVC estimator。
print("Test set score:{:.2f}".format(grid_search.score(X_test,y_test)))
print("Best parameters:{}".format(grid_search.best_params_))
print("Best score on train set:{:.2f}".format(grid_search.best_score_))

------
参数说明： 

class sklearn.model_selection.GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
1. estimator：estimator object.This is assumed to implement the scikit-learn estimator interface. 
2. cv：None, to use the default 5-fold cross validation；integer, to specify the number of folds in a (Stratified)KFold。
3. verbose：verbose：日志冗长度，int：冗长度，0：不输出训练过程，1：偶尔输出，>1：对每个子模型都输出。>1 : the computation time for each fold and parameter candidate is displayed;>2 : the score is also displayed;>3 : the fold and candidate parameter indexes are also displayed together with the starting time of the computation.
------